# Import Require Libary

In [1]:
import pandas as pd
import numpy as np
import os

#### ** Create Some Modification on each sheet and create function **

In [2]:
# Change B2B Values
def B2B(df):
    df.drop([0,1,2],inplace=True)
    df.loc[4][ df.loc[4].isnull()] =  df.loc[3]
    df.drop(3,inplace= True)
    df.columns = df.iloc[0].values
    df.drop(4,inplace= True)
    return df
    
# Change B2BA Values
def B2BA(df):
    df.iloc[5][df.iloc[5].isnull()] = df.iloc[4]
    df.drop([0,1,2,3,4],inplace= True)
    df.columns = df.iloc[0]
    df.drop(5,inplace=True)
    return df

def B2BCDNR(df):
    df.loc[4][df.loc[4].isnull()] = df.loc[3]
    df.drop([0,1,2,3],inplace=True)
    df.columns = df.loc[4]
    df.drop(4,inplace=True)
    return df

def IMPG(df):
    df.loc[4][df.loc[4].isnull()]= df.loc[3]
    df.drop([0,1,2,3],inplace=True)
    df.columns= df.iloc[0]
    df.drop([4],inplace=True)
    return df

def IMPGSEZ(df):
    df.loc[4][df.loc[4].isnull()] = df.loc[3]
    df.columns= df.iloc[4]
    df.drop([0,1,2,3,4],inplace=True)
    return df

def CDNRA(df):
    df.loc[5] [df.loc[5].isnull()]= df.loc[4]
    df.columns= df.loc[5]
    df.drop([0,1,2,3,4,5],inplace= True)
    return df

In [3]:
def MergingB2B_B2BA(B2Bs,B2BAs):
       # For Merging of B2BA and B2B
       B2BAs.columns.values[4]= 'New ' + B2BAs.columns.values[0]
       B2BAs.columns.values[6]= 'New ' + B2BAs.columns.values[1]
       ds = pd.merge(B2Bs,B2BAs,how='outer',on=['GSTIN of supplier','Invoice number','Invoice Date'])

       # For Select Invoice NumBer
       Values = [ds['New Invoice number'] ,ds['Invoice number'] ]
       cod =[ds['New Invoice number'].notnull(),ds['New Invoice number'].isnull()]
       ds['Invoice number'] =np.select(cod,Values)
       # ds['Invoice number'] = ds.apply(lambda x: x['New Invoice number'] if ['New Invoice number'].notnull() else x['Invoice number'],axis=1)


       # For Select Invoice Date
       Values = [ds['New Invoice Date'] ,ds['Invoice Date'] ]
       cod =[ds['New Invoice number'].notnull(),ds['New Invoice number'].isnull()]
       ds['Invoice Date'] =np.select(cod,Values)


       # For All Adjustment
       for x in ['Trade/Legal name','Invoice Value(₹)','Place of supply','Supply Attract Reverse Charge','Rate(%)','Taxable Value (₹)','Integrated Tax(₹)','Central Tax(₹)','State/UT Tax(₹)','Cess(₹)','ITC Availability','Source']:
              Values = [ds[x+'_y'] ,ds[x+'_x'] ]
              cod =[ds['New Invoice number'].notnull(),ds['New Invoice number'].isnull()]
              ds[x] =np.select(cod,Values)
              

       # Removing All other Columns
       ds.drop(ds.columns.difference(['GSTIN of supplier', 'Invoice number', 'Invoice Date',
              'Trade/Legal name', 'Invoice Value(₹)', 'Place of supply',
              'Supply Attract Reverse Charge', 'Rate(%)', 'Taxable Value (₹)',
              'Integrated Tax(₹)', 'Central Tax(₹)', 'State/UT Tax(₹)', 'Cess(₹)',
              'ITC Availability','Source']) ,axis=1, inplace=True)
       
       B2Bs= ds
       return B2Bs

       # ds

In [4]:
path = '2B-Data\\'
ds= []
L1= ['B2B','B2BA','B2B-CDNR','B2B-CDNRA','IMPG','IMPGSEZ']
df= {'B2B':[],'B2BA':[],'B2B-CDNR':[],'B2B-CDNRA':[],'IMPG':[],'IMPGSEZ':[]}
s=set()
for x in L1:
    for a,i in enumerate( os.listdir(path)):
            # Avoid error if worksheet not avaialabe
            if x in pd.ExcelFile(path+i).sheet_names:
                ds = pd.read_excel(path+i,x)
                # Add Source Columns
                if x== L1[0]:
                    B2B(ds)
                    ds['Source'] =x
                    df[x].append(ds)
                elif x== L1[1]:
                    B2BA(ds)
                    ds['Source'] ='B2BA'
                    df[x].append(ds)
                elif x== L1[2]:
                    B2BCDNR(ds)
                    ds['Source'] ='CDNR'
                    df[x].append(ds)
                elif x== L1[3]:
                    CDNRA(ds)
                    ds['Source'] ='CDNRA'
                    df[x].append(ds)
                elif x== L1[4]:
                    IMPG(ds)
                    ds['Source'] ='IMPG'
                    df[x].append(ds)
                elif x== L1[5]:
                    IMPGSEZ(ds)
                    ds['Source'] ='IMPGSEZ'
                    df[x].append(ds)
                ds= []
    df[x]= pd.concat(df[x])
    

In [5]:
def MergingCDNRA_CDNRA(CDNRs,CDNRAs):
       # For Merging of B2BA and B2B
       CDNRAs.columns.values[0]= 'New ' + CDNRAs.columns.values[5]
       CDNRAs.columns.values[1]= 'New ' + CDNRAs.columns.values[8]
       CDNRAs.columns.values[2]= 'New ' + CDNRAs.columns.values[6]
       ds = pd.merge(CDNRs,CDNRAs,how='outer',on=['GSTIN of supplier','Note number','Note date','Note type'])

       # For Select Invoice NumBer
       Values = [ds['New Note number'] ,ds['Note number'] ]
       cod =[ds['New Note number'].notnull(),ds['New Note number'].isnull()]
       ds['Invoice number'] =np.select(cod,Values)
       # # ds['Invoice number'] = ds.apply(lambda x: x['New Invoice number'] if ['New Invoice number'].notnull() else x['Invoice number'],axis=1)


       # For Select Invoice Date
       Values = [ds['New Note type'] ,ds['Note type'] ]
       cod =[ds['New Note type'].notnull(),ds['New Note type'].isnull()]
       ds['Invoice Date'] =np.select(cod,Values)

       # For Select Invoice Date
       Values = [ds['New Note date'] ,ds['Note date'] ]
       cod =[ds['New Note date'].notnull(),ds['New Note date'].isnull()]
       ds['Invoice Type'] =np.select(cod,Values)


       # For All Adjustment
       for x in ['Trade/Legal name','Place of supply','Supply Attract Reverse Charge',
       'Rate(%)','Taxable Value (₹)','Integrated Tax(₹)','Central Tax(₹)','State/UT Tax(₹)',
       'Cess(₹)','Source','ITC Availability']:
              Values = [ds[x+'_y'] ,ds[x+'_x'] ]
              cod =[ds['New Note number'].notnull(),ds['New Note number'].isnull()]
              ds[x] =np.select(cod,Values)
              

       # Removing All other Columns
       ds.drop(ds.columns.difference(['Trade/Legal name','Place of supply',
       'Supply Attract Reverse Charge','Rate(%)','Taxable Value (₹)','Integrated Tax(₹)',
       'Central Tax(₹)','State/UT Tax(₹)','Cess(₹)','Source','ITC Availability',
       'New Note date','Invoice Date','Invoice number']) ,axis=1, inplace=True)
       
       CDNRs= ds
       return CDNRs

In [6]:
df[L1[0]] =MergingB2B_B2BA(df[L1[0]],df[L1[1]])
df[L1[2]] =MergingCDNRA_CDNRA(df[L1[2]],df[L1[3]])

In [20]:
df[L1[2]].columns

Index(['New Note date', 'Invoice number', 'Invoice Date', 'Trade/Legal name',
       'Place of supply', 'Supply Attract Reverse Charge', 'Rate(%)',
       'Taxable Value (₹)', 'Integrated Tax(₹)', 'Central Tax(₹)',
       'State/UT Tax(₹)', 'Cess(₹)', 'Source', 'ITC Availability'],
      dtype='object')